# Empresas representadas por el declarante, pareja o dependientes y el RUPC (4_2_06)
| **Escenario**                                  | **Condición exacta evaluada**                                                                                        | **Resultado**    |
| ---------------------------------------------- | -------------------------------------------------------------------------------------------------------------------- | ---------------- |
| **No hay representaciones declaradas**         | La lista `representaciones.representacion` no existe o viene vacía                                                   | ⚪ **NO_APLICA**  |
| **Faltan datos esenciales**                    | Falta: <br>• nombre de empresa representada, o <br>• indicador de representación (“Representante” + “Persona moral”) | ⚪ **SIN_DATO**   |
| **La empresa representada aparece en el RUPC** | Normalización coincide con razón social de RUPC                                                                      | 🔴 **NO_CUMPLE** |
| **La empresa NO aparece en el RUPC**           | Nombre normalizado **no coincide** con ninguna empresa del RUPC                                                      | 🟢 **CUMPLE**    |
| **Campos presentes pero incompletos o nulos**  | Representación declarada sin empresa válida                                                                          | ⚪ **SIN_DATO**   |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME
import unicodedata
import openpyxl
import os

METRIC_ID = "4_2_06_REPRESENTACION_Y_RUPC"
RUPC_FILE = "rupc.xlsx"


# -------------------------------------------------------------
# AUXILIARES DE TEXTO
# -------------------------------------------------------------
def normalizar_texto(v):
    if not v:
        return None
    v = str(v).strip().upper()
    v = v.replace('"', "").replace("'", "")
    v = ''.join(c for c in unicodedata.normalize('NFD', v)
                if unicodedata.category(c) != 'Mn')
    while "  " in v:
        v = v.replace("  ", " ")
    return v


def extraer(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p)
            else:
                return None
        return actual
    except:
        return None


# -------------------------------------------------------------
# LECTURA DEL RUPC DESDE EXCEL
# -------------------------------------------------------------
def cargar_rupc():
    empresas = set()

    if not os.path.exists(RUPC_FILE):
        print(f"❌ No se encontró el archivo {RUPC_FILE}")
        return empresas

    try:
        print(f"📄 Leyendo archivo RUPC: {RUPC_FILE}")
        wb = openpyxl.load_workbook(RUPC_FILE, read_only=True, data_only=True)
        ws = wb.active

        # encabezado está en fila 2 (la 1 trae “Última actualización…”)
        header_raw = [c.value for c in next(ws.iter_rows(min_row=2, max_row=2))]
        header = [normalizar_texto(c) for c in header_raw]

        # buscar columna con NOMBRE DE EMPRESA
        idx_nombre = None
        for i, h in enumerate(header):
            if "NOMBRE" in h and "EMPRESA" in h:
                idx_nombre = i + 1
                break

        if idx_nombre is None:
            print("❌ No se encontró columna 'Nombre de la empresa'")
            print(header)
            return empresas

        for row in ws.iter_rows(min_row=3):
            celda = row[idx_nombre - 1].value
            nom = normalizar_texto(celda)
            if nom:
                empresas.add(nom)

    except Exception as e:
        print("❌ Error cargando RUPC:", e)

    print(f"🔎 Empresas RUPC cargadas: {len(empresas)}")
    return empresas


# -------------------------------------------------------------
# EVALUACIÓN DE LA MÉTRICA
# -------------------------------------------------------------
def evaluar_metrica(doc, empresas_rupc):

    reps = extraer(doc, "declaracion.intereses.representaciones.representacion")

    if not reps:
        return "NO_APLICA"

    if not isinstance(reps, list):
        reps = [reps]

    hubo_datos = False

    for rep in reps:

        # Validar si marcó Representante + Persona moral
        es_rep = normalizar_texto(rep.get("representante"))
        es_pm = normalizar_texto(rep.get("personaMoral"))

        if es_rep != "SI" or es_pm != "SI":
            continue  # no aplica para este registro

        nombre = normalizar_texto(rep.get("nombre"))  # <<< Ajustar si tu campo se llama diferente

        if not nombre:
            return "SIN_DATO"

        hubo_datos = True

        # Verificar si empresa está en el RUPC
        if nombre in empresas_rupc:
            return "NO_CUMPLE"

    if not hubo_datos:
        return "NO_APLICA"

    return "CUMPLE"


# -------------------------------------------------------------
# MOTOR MONGO
# -------------------------------------------------------------
def procesar_metrica_4_2_06():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "NO_APLICA": 0}
    operaciones = []
    total = 0

    try:
        print(f"\n🚀 Procesando métrica {METRIC_ID}...\n")

        empresas_rupc = cargar_rupc()

        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]

        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.intereses.representaciones": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1

            try:
                resultado = evaluar_metrica(doc, empresas_rupc)
            except Exception as e:
                print(f"⚠️ Error en doc {doc.get('_id')}: {e}")
                resultado = "SIN_DATO"

            resultados[resultado] += 1

            operaciones.append(
                UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}},
                    upsert=True
                )
            )

            if len(operaciones) >= 2000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"> {total} documentos procesados...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Procesados: {total}")
        for k, v in resultados.items():
            print(f"  {k}: {v}")

    except:
        traceback.print_exc()

    finally:
        try:
            client.close()
        except:
            pass

        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_4_2_06()



🚀 Procesando métrica 4_2_06_REPRESENTACION_Y_RUPC...

📄 Leyendo archivo RUPC: rupc.xlsx
🔎 Empresas RUPC cargadas: 24850


c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


> 2000 documentos procesados...
> 4000 documentos procesados...
> 6000 documentos procesados...
> 8000 documentos procesados...
> 10000 documentos procesados...
> 12000 documentos procesados...
> 14000 documentos procesados...
> 16000 documentos procesados...
> 18000 documentos procesados...
> 20000 documentos procesados...
> 22000 documentos procesados...
> 24000 documentos procesados...
> 26000 documentos procesados...
> 28000 documentos procesados...
> 30000 documentos procesados...
> 32000 documentos procesados...
